## 참고
- <https://minimin2.tistory.com/137>
- <https://leo-bb.tistory.com/62>
- <https://dacon.io/codeshare/2430>

In [1]:
import pycaret

In [2]:
from pycaret.datasets import get_data
data = get_data('bank')

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [3]:
from pycaret.classification import *

model = setup(
    data = data,
    target = 'deposit'
)

,Description,Value
0,session_id,6845
1,Target,deposit
2,Target Type,Binary
3,Label Encoded,"no: 0, yes: 1"
4,Original Data,"(45211, 17)"
5,Missing Values,False
6,Numeric Features,7
7,Categorical Features,9
8,Ordinal Features,False
9,High Cardinality Features,False


In [4]:
# 모델을 생성
my_catboost = create_model(
    estimator = 'catboost',
    fold = 5,
    round = 4,
    cross_validation = True,
    verbose = True,
    system = True,
)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9107,0.9391,0.4932,0.6523,0.5617,0.5131,0.5193
1,0.9060,0.9351,0.4646,0.6280,0.5341,0.4831,0.4899
2,0.9163,0.9395,0.5116,0.6856,0.5859,0.5404,0.5476
3,0.9107,0.9374,0.4802,0.6567,0.5548,0.5065,0.5142
4,0.9085,0.9362,0.4850,0.6391,0.5515,0.5016,0.5076
Mean,0.9104,0.9375,0.4869,0.6523,0.5576,0.5089,0.5157
SD,0.0034,0.0017,0.0155,0.0194,0.0168,0.0186,0.0188


In [5]:
my_catboost

In [6]:
# 역시나 모델을 생성하지만 `create_model`과 다르게 가능한 모든 모델을 생성하며, 모두 성능을 비교하여 가장 좋은 모델을 뽑을 수 있음.
top5 = compare_models(
    exclude = None,
    include = None,
    fold = 5,
    round = 4,
    sort = 'AUC',
    n_select = 5,
    budget_time = 0,
    turbo = True,
    verbose = True
)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.9104,0.9375,0.4869,0.6523,0.5576,0.5089,0.5157,4.5840
lightgbm,Light Gradient Boosting Machine,0.9091,0.9352,0.4828,0.6444,0.5519,0.5025,0.5091,0.1080
xgboost,Extreme Gradient Boosting,0.9068,0.9321,0.4804,0.6279,0.5442,0.4933,0.4988,37.0120
rf,Random Forest Classifier,0.9058,0.9269,0.3650,0.6734,0.4731,0.4264,0.4507,0.4420
gbc,Gradient Boosting Classifier,0.9067,0.9253,0.4076,0.6575,0.5029,0.4546,0.4706,0.6880
ada,Ada Boost Classifier,0.9000,0.9106,0.3710,0.6140,0.4622,0.4108,0.4268,0.2260
lda,Linear Discriminant Analysis,0.9008,0.9093,0.4381,0.5992,0.5059,0.4522,0.4592,0.0640
et,Extra Trees Classifier,0.9010,0.9091,0.3367,0.6384,0.4405,0.3919,0.4163,0.4240
lr,Logistic Regression,0.9016,0.9036,0.3438,0.6409,0.4470,0.3984,0.4220,1.1180
nb,Naive Bayes,0.8568,0.8234,0.5196,0.4078,0.4569,0.3759,0.3796,0.1840


In [7]:
top5

 LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                importance_type='split', learning_rate=0.1, max_depth=-1,
                min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
                n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
                random_state=6845, reg_alpha=0.0, reg_lambda=0.0, silent=True,
                subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
 XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
               importance_type='gain', interaction_constraints='',
               learning_rate=0.300000012, max_delta_step=0, max_depth=6,
               min_child_weight=1, missing=nan, monotone_constraints='()',
               n_estimators=100, n_jobs=-1, num_parallel_tree=1,
               objective='binary:logistic', random_state=6845, reg_alpha=0,
               reg_lambda=1, scal

In [8]:
tuned_top5 = [tune_model(
    estimator = model,
    fold = 5,
    round = 4,
    n_iter = 10,
    custom_grid = None,  # 커스텀 파라미터 dict
    optimize = 'AUC',
    custom_scorer = None,
    choose_better = False,
    verbose = True
) for model in top5]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9081,0.9312,0.4142,0.6667,0.5109,0.4632,0.4792
1,0.9058,0.9262,0.4005,0.6533,0.4966,0.4480,0.4643
2,0.9079,0.9310,0.3970,0.6736,0.4996,0.4525,0.4718
3,0.9049,0.9293,0.3752,0.6563,0.4774,0.4294,0.4497
4,0.9061,0.9275,0.3924,0.6606,0.4923,0.4443,0.4626
Mean,0.9066,0.9291,0.3959,0.6621,0.4954,0.4475,0.4655
SD,0.0012,0.0019,0.0126,0.0073,0.0109,0.0111,0.0099


In [9]:
# 이전 `top_5`와는 parameter가 달라진 것을 볼 수 있다.
tuned_top5

 LGBMClassifier(bagging_fraction=0.6, bagging_freq=7, boosting_type='gbdt',
                class_weight=None, colsample_bytree=1.0, feature_fraction=0.6,
                importance_type='split', learning_rate=0.2, max_depth=-1,
                min_child_samples=41, min_child_weight=0.001, min_split_gain=0.1,
                n_estimators=200, n_jobs=-1, num_leaves=10, objective=None,
                random_state=6845, reg_alpha=2, reg_lambda=1, silent=True,
                subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
 XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
               importance_type='gain', interaction_constraints='',
               learning_rate=0.05, max_delta_step=0, max_depth=6,
               min_child_weight=3, missing=nan, monotone_constraints='()',
               n_estimators=130, n_jobs=-1, num_parallel_tree=1,
               objective='binary:logistic', 

In [10]:
# 5개 모델을 혼합한 모델
blender_top5 = blend_models(estimator_list=tuned_top5)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9096,0.9392,0.5559,0.6239,0.5879,0.5373,0.5385
1,0.9093,0.9383,0.6349,0.6036,0.6189,0.5674,0.5677
2,0.9062,0.9373,0.5559,0.6036,0.5787,0.5260,0.5266
3,0.9065,0.9303,0.5804,0.6000,0.5900,0.5373,0.5374
4,0.9125,0.9343,0.6076,0.6264,0.6169,0.5675,0.5676
5,0.9153,0.9405,0.6022,0.6443,0.6225,0.5749,0.5753
6,0.9112,0.9367,0.5967,0.6222,0.6092,0.5591,0.5593
7,0.9090,0.9376,0.5765,0.6134,0.5944,0.5432,0.5435
8,0.9083,0.9269,0.5546,0.6152,0.5833,0.5320,0.5329
9,0.9147,0.9458,0.6322,0.6322,0.6322,0.5839,0.5839


In [11]:
# K-fold가 아닌 전체 데이터로 한번에 학습
final_model = finalize_model(blender_top5)

In [13]:
prediction = predict_model(final_model, data=data.iloc[-100:])

In [14]:
prediction

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit,Label,Score
45111,64,retired,married,primary,no,445,no,no,cellular,26,oct,338,3,186,1,other,no,yes,0.5456
45112,24,student,single,secondary,no,1234,no,no,telephone,26,oct,280,2,97,3,success,no,yes,0.7612
45113,26,management,single,tertiary,no,222,no,no,cellular,26,oct,318,1,120,3,success,yes,yes,0.8181
45114,36,technician,single,secondary,no,204,yes,no,cellular,26,oct,196,1,144,8,other,no,yes,0.5269
45115,30,management,single,tertiary,no,565,yes,no,cellular,26,oct,104,1,-1,0,unknown,no,no,0.6622
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes,yes,0.6222
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes,yes,0.6257
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes,yes,0.7944
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no,no,0.7420


In [17]:
from pycaret.utils import check_metric
check_metric(prediction['deposit'], prediction['Label'], metric = 'AUC')

0.7532